# Модель прогнозування продажів товарів
## Model_pcs_accuracy_calculation

In [ ]:
# Імпорт бібліотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Створення синтетичного датасету для прикладу
np.random.seed(42)
n_samples = 1000

data = {
    'product_id': np.random.randint(1, 50, n_samples),
    'month': np.random.randint(1, 13, n_samples),
    'day_of_week': np.random.randint(0, 7, n_samples),
    'price': np.random.uniform(10, 500, n_samples),
    'promotion': np.random.choice([0, 1], n_samples, p=[0.7, 0.3]),
    'competitor_price': np.random.uniform(10, 500, n_samples),
    'stock_level': np.random.randint(0, 200, n_samples)
}

# Генерація цільової змінної (кількість продажів)
df = pd.DataFrame(data)
df['sales_quantity'] = (
    100 - 0.3 * df['price'] + 
    50 * df['promotion'] + 
    0.2 * df['stock_level'] +
    np.random.normal(0, 10, n_samples)
).clip(0)

print("Датасет створено:")
print(df.head())
print(f"\nРозмір датасету: {df.shape}")

In [ ]:
# Підготовка даних
X = df.drop('sales_quantity', axis=1)
y = df['sales_quantity']

# Розділення на тренувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Тренувальна вибірка: {X_train.shape}")
print(f"Тестова вибірка: {X_test.shape}")

In [ ]:
# Навчання моделі Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

print("Модель навчена успішно!")
print(f"\nВажливість ознак:")
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)
print(feature_importance)

In [ ]:
# Прогнозування
y_pred = model.predict(X_test)

# Створення DataFrame з результатами
results_df = pd.DataFrame({
    'actual': y_test.values,
    'predicted': y_pred
})

print("Перші 10 прогнозів:")
print(results_df.head(10))

## Блок обчислення метрик (MAE, MAPE, WMAPE)

In [ ]:
# Обчислення метрик точності
from sklearn.metrics import mean_absolute_error

# MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)

# MAPE (Mean Absolute Percentage Error)
# Уникаємо ділення на нуль
mask = y_test != 0
mape = np.mean(np.abs((y_test[mask] - y_pred[mask]) / y_test[mask])) * 100

# WMAPE (Weighted Mean Absolute Percentage Error)
wmape = (np.sum(np.abs(y_test - y_pred)) / np.sum(np.abs(y_test))) * 100

print("=" * 50)
print("МЕТРИКИ ТОЧНОСТІ МОДЕЛІ")
print("=" * 50)
print(f"MAE (Mean Absolute Error): {mae:.2f}")
print(f"MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
print(f"WMAPE (Weighted Mean Absolute Percentage Error): {wmape:.2f}%")
print("=" * 50)

In [ ]:
# Створення CSV-файлу з результатами
output_data = {
    'Model': ['Product Sales Forecast - Random Forest'],
    'MAE': [mae],
    'MAPE': [mape],
    'WMAPE': [wmape],
    'Test_Samples': [len(y_test)],
    'Date': [pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')]
}

output_df = pd.DataFrame(output_data)
output_filename = 'pcs_accuracy_calculation.csv'
output_df.to_csv(output_filename, index=False)

print(f"\n✓ Результати збережено у файл: {output_filename}")
print("\nВміст файлу:")
print(output_df)

In [ ]:
# Візуалізація результатів
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Фактичні продажі')
plt.ylabel('Прогнозовані продажі')
plt.title('Порівняння фактичних та прогнозованих продажів')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✓ Модель успішно навчена і протестована!")